In [1]:
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import *
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import *
from OCC.Core.BRepBuilderAPI import *
from math import pi,degrees,radians
from OCC.Core.BRepTools import breptools_Write
import OCCUtils
import OCCUtils.edge
import numpy as np
import OCCUtils.face
from OCC.Core.BRepAlgoAPI import *
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh

In [2]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [4]:
import sys
sys.path.append("/home/adam/projects/FreeCAD/build/Mod/Path")
import area

In [5]:
a2d = area.Adaptive2d()

In [6]:
a2d.tolerance

0.1

In [7]:
a2d.opType

area.AdaptiveOperationType.ClearingInside

In [8]:
a2d.stockToLeave

0.0

In [9]:
a2d.toolDiameter = 10

In [10]:
a2d.helixRampDiameter = 3

In [11]:
a2d.stepOverFactor = 0.5

In [12]:
def callback(something):
    return False # True stops processing

In [13]:
stock_path = [[[0,0],[100,0],[100,100],[0,100]]]

In [14]:
path = [[[10,10],[90,10],[90,90],[10,90]]]

In [15]:
output = a2d.Execute(stock_path,path,callback)

In [16]:
output

In [17]:
for o in output:
    print(o.HelixCenterPoint)
    print(o.AdaptivePaths)
    print('return motion type:{}'.format(o.ReturnMotionType))

(50.0, 49.99375)
[(0, [(50.0, 48.49375), (50.06875, 48.41875), (50.1625, 48.2375), (50.30625, 48.1), (50.475, 48.0), (50.84375, 47.875), (51.125, 47.8375), (51.8625, 47.9875), (52.16875, 48.14375), (52.55625, 48.425), (52.83125, 48.71875), (53.05625, 49.05625), (53.2, 49.3625), (53.29375, 49.65625), (53.36875, 49.99375), (53.3875, 50.45), (53.3625, 50.80625), (53.29375, 51.1875), (53.16875, 51.5625), (53.00625, 51.90625), (52.79375, 52.23125), (52.5625, 52.55), (52.18125, 52.9375), (51.85, 53.2125), (51.45625, 53.4375), (51.1375, 53.58125), (50.74375, 53.7), (50.34375, 53.7625), (49.9625, 53.8), (49.43125, 53.775), (49.00625, 53.7125), (48.58125, 53.58125), (48.2375, 53.4375), (47.525, 53.00625), (47.03125, 52.58125), (46.7375, 52.21875), (46.54375, 51.93125), (46.225, 51.2375), (46.075, 50.65625), (46.00625, 50.225), (45.975, 49.76875), (46.0, 49.28125), (46.10625, 48.8375), (46.20625, 48.50625), (46.35, 48.1125), (46.64375, 47.5), (46.98125, 46.9375), (47.3375, 46.50625), (47.6125, 4

In [18]:
# contents of AdaptivePaths is a list of tuples.
# [(connection_type,[(x,y),...,(x,y)]),
#  (connection_type),[(x,y),...,(x,y)]]
#
# connection_type = 0 means continue from previous feed rate at horizontal feedrate ?
# connection_type = 1 means do a rapid motion from where you were to the next point ?

In [19]:
o.HelixCenterPoint

(50.0, 49.99375)

In [20]:
points = output[0].AdaptivePaths[0][1]

0 means continue at feedrate

1 means rapid to here

In [23]:
output[0].AdaptivePaths

[(0,
  [(50.0, 48.49375),
   (50.06875, 48.41875),
   (50.1625, 48.2375),
   (50.30625, 48.1),
   (50.475, 48.0),
   (50.84375, 47.875),
   (51.125, 47.8375),
   (51.8625, 47.9875),
   (52.16875, 48.14375),
   (52.55625, 48.425),
   (52.83125, 48.71875),
   (53.05625, 49.05625),
   (53.2, 49.3625),
   (53.29375, 49.65625),
   (53.36875, 49.99375),
   (53.3875, 50.45),
   (53.3625, 50.80625),
   (53.29375, 51.1875),
   (53.16875, 51.5625),
   (53.00625, 51.90625),
   (52.79375, 52.23125),
   (52.5625, 52.55),
   (52.18125, 52.9375),
   (51.85, 53.2125),
   (51.45625, 53.4375),
   (51.1375, 53.58125),
   (50.74375, 53.7),
   (50.34375, 53.7625),
   (49.9625, 53.8),
   (49.43125, 53.775),
   (49.00625, 53.7125),
   (48.58125, 53.58125),
   (48.2375, 53.4375),
   (47.525, 53.00625),
   (47.03125, 52.58125),
   (46.7375, 52.21875),
   (46.54375, 51.93125),
   (46.225, 51.2375),
   (46.075, 50.65625),
   (46.00625, 50.225),
   (45.975, 49.76875),
   (46.0, 49.28125),
   (46.10625, 48.8375),


In [21]:
edges = []
mw = BRepBuilderAPI_MakeWire()
p_i = points.pop(0)
p_0 = gp_Pnt(points[0][0],points[0][1],0)

good_edges = 0
failed_edges = 0
for p in points:
    p_1 = gp_Pnt(p[0],p[1],0)
    try:
        me = BRepBuilderAPI_MakeEdge(p_0,p_1)
        edge = me.Edge()
        edge_success = True
    except:
        failed_edges += 1
        edge_success = False
        pass
    if edge_success:
        good_edges += 1
        p_0 = p_1
        mw.Add(edge)

points.insert(0,p_i)
wire = mw.Wire()
print("good edges:",good_edges)
print("failed edges:",failed_edges)

good edges: 1034
failed edges: 1


In [22]:
breptools_Write(wire,"/tmp/wire.brep")

True

In [23]:
renderer = JupyterRenderer()

In [24]:
renderer.DisplayShape(wire)

In [25]:
renderer

# Practicing coordinate system transformations

In [29]:
ax = gp_Ax2(gp_Pnt(0,30,0),
            gp_Dir(-1,1,0),
            gp_Dir(1,0,0)
           )
pln = gp_Pln(gp_Ax3(ax))
mf = BRepBuilderAPI_MakeFace(pln,-20,20,-10,10)
face = mf.Face()

In [30]:
renderer.DisplayShape(face)

In [31]:
# finding the transformation matrix from main c.s. to c.s. in ax

In [32]:
v_origin = gp_Vec(ax.Location().XYZ())

In [33]:
mat = gp_Mat() # rotation from B to main c.s.
mat.SetRows(ax.XDirection().XYZ(),
       ax.YDirection().XYZ(),
       ax.Direction().XYZ()
       )

In [34]:
R = mat.Inverted()

In [35]:
v_origin_mat = gp_Mat()
v_origin_mat.SetRow(1,v_origin.XYZ())

In [36]:
v_T_inv = gp_Vec(gp_XYZ((v_origin_mat * R).Value(1,1),
                        (v_origin_mat * R).Value(1,2),
                        (v_origin_mat * R).Value(1,3)))

In [37]:
r_to_b = mat.Inverted() # rotation matrix from main c.s. to b c.s.

In [38]:
trsf = gp_Trsf()
trsf.SetValues(mat.Value(1,1),mat.Value(1,2),mat.Value(1,3),-v_T_inv.X(),
               mat.Value(2,1),mat.Value(2,2),mat.Value(2,3),-v_T_inv.Y(),
               mat.Value(3,1),mat.Value(3,2),mat.Value(3,3),-v_T_inv.Z())

In [39]:
mt = BRepBuilderAPI_Transform(face,trsf)

In [40]:
face_transformed = mt.Shape()

In [41]:
renderer.DisplayShape(face_transformed)

In [40]:
square_pacman = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(square_pacman, './square_pacman.brep', builder)
t = OCCUtils.Topo(square_pacman)
square_pacman = t.faces().__next__()
square_pacman.Reverse()

In [41]:
renderer.DisplayShape(square_pacman)

In [52]:
f = OCCUtils.face.Face(square_pacman)

In [53]:
pln = f.adaptor.Plane()

In [54]:
ax_on_sqp = gp_Ax2(pln.Location(),pln.Axis().Direction(),pln.XAxis().Direction())

In [55]:
ax_on_sqp.Rotate(gp_Ax1(ax_on_sqp.Location(),ax_on_sqp.XDirection()),pi)

In [46]:
gp_Vec(ax_on_sqp.Direction().XYZ())

< gp_Vec: 1.5530666861673522e-16, -0.3255681544571567, 0.945518575599317, magnitude: 1.0 >

# Drawing a Vector in space

In [42]:
from OCC.Core.BRepPrimAPI import *

In [44]:
class VectorGlyph:
    def __init__(self,v_tip,v_tail):
        self.v_tip = v_tip
        self.v_tail = v_tail
        self.shaft_diameter = 4
        self.cone_diameter = 3*self.shaft_diameter
        self.cone_length = 4*self.shaft_diameter
    def Shape(self):
        length = (self.v_tip - self.v_tail).Magnitude()
        cylinder_length = length - self.cone_length
        direction = gp_Dir((self.v_tip-self.v_tail).XYZ())
        ax = gp_Ax2(gp_Pnt(self.v_tail.XYZ()),direction)
        cylinder = False
        if cylinder_length > 0:
            mc = BRepPrimAPI_MakeCylinder(ax,self.shaft_diameter/2,length-self.cone_length)
            cylinder = mc.Shape()
            
        # make cone
        v_cone_base = self.v_tip + (self.v_tail - self.v_tip).Normalized()*self.cone_length
        ax = gp_Ax2(gp_Pnt(v_cone_base.XYZ()),direction)
        mcone = BRepPrimAPI_MakeCone(ax,self.cone_diameter/2,0,self.cone_length)
        cone = mcone.Shape()
        
        if cylinder:
            mf = BRepAlgoAPI_Fuse(cone,cylinder)
            glyph = mf.Shape()
        else:
            glyph = cone
        
        return glyph
        

In [49]:
v_tail = gp_Vec(ax_on_sqp.Location().XYZ())
v_tip = v_tail + gp_Vec(ax_on_sqp.Direction().XYZ())*100
vg = VectorGlyph(v_tip,v_tail)

In [50]:
vg_shape = vg.Shape()
vg_shape

<class 'TopoDS_Compound'>

In [51]:
renderer.DisplayShape(vg.Shape())

# Drawing an Ax2

In [45]:
class Ax2Glyph:
    
    def __init__(self,ax2):
        self.ax = ax2
        self.vectorLengths = (50,50,100)
        
    def Shape(self):
        v_origin = gp_Vec(self.ax.Location().XYZ())
        # draw the z axis
        v_zdir = gp_Vec(self.ax.Direction().XYZ())
        v_tip_z = v_origin + v_zdir*self.vectorLengths[2]
        vg_z = VectorGlyph(v_tip_z,v_origin)
        
        #draw the x axis
        v_xdir = gp_Vec(self.ax.XDirection().XYZ())
        v_tip_x = v_origin + v_xdir*self.vectorLengths[0]
        vg_x = VectorGlyph(v_tip_x,v_origin)
        
        #draw the y axis
        v_ydir = gp_Vec(self.ax.YDirection().XYZ())
        v_tip_y = v_origin + v_ydir*self.vectorLengths[1]
        vg_y = VectorGlyph(v_tip_y,v_origin)
        
        mfuse = BRepAlgoAPI_Fuse(vg_x.Shape(),vg_y.Shape())
        shape = mfuse.Shape()
        mfuse = BRepAlgoAPI_Fuse(shape,vg_z.Shape())
        
        return mfuse.Shape()

In [53]:
vg_ax = Ax2Glyph(ax_on_sqp)

In [54]:
vg_ax_shape = vg_ax.Shape()

In [55]:
breptools_Write(vg_ax_shape,"/tmp/ax.brep")

True

In [56]:
renderer.DisplayShape(vg_ax_shape)

# Developing a function to make gp_Trsf's from an Ax2

In [46]:
def trsf_from_Ax2(ax2):
    
    # finds the transformation matrix from main c.s. to c.s. in ax2
    v_origin = gp_Vec(ax2.Location().XYZ())

    mat = gp_Mat() # rotation from B to main c.s.
    mat.SetRows(
           ax2.XDirection().XYZ(),
           ax2.YDirection().XYZ(),
           ax2.Direction().XYZ()
           )

    R = mat.Inverted()

    v_origin_mat = gp_Mat()
    v_origin_mat.SetRow(1,v_origin.XYZ())

    v_T_inv = gp_Vec(gp_XYZ((v_origin_mat * R).Value(1,1),
                            (v_origin_mat * R).Value(1,2),
                            (v_origin_mat * R).Value(1,3)))

    r_to_b = mat.Inverted() # rotation matrix from main c.s. to b c.s.

    trsf = gp_Trsf()
    trsf.SetValues(mat.Value(1,1),mat.Value(1,2),mat.Value(1,3),-v_T_inv.X(),
                   mat.Value(2,1),mat.Value(2,2),mat.Value(2,3),-v_T_inv.Y(),
                   mat.Value(3,1),mat.Value(3,2),mat.Value(3,3),-v_T_inv.Z())
    return trsf

In [58]:
trsf = trsf_from_Ax2(ax_on_sqp)

In [59]:
mt = BRepBuilderAPI_Transform(square_pacman,trsf)
square_pacman_transformed = mt.Shape()

In [60]:
renderer.DisplayShape(square_pacman_transformed)

In [61]:
mt = BRepBuilderAPI_Transform(square_pacman_transformed,trsf.Inverted())
square_pacman_untransformed = mt.Shape()

In [62]:
renderer.DisplayShape(square_pacman_untransformed)

# Get tesscilation of wire
(not successfull)

In [67]:
wire = OCCUtils.Topo(square_pacman_transformed).wires().__next__()

In [68]:
im = BRepMesh_IncrementalMesh(wire,0.1)

In [69]:
import OCC.Core.Tesselator

In [70]:
tess = OCC.Core.Tesselator.ShapeTesselator(wire)

In [71]:
tess.Compute(compute_edges=True,mesh_quality=0.001)

In [72]:
tess.ObjGetEdgeCount()

0

# Get tesselation of edge
(not successful)

In [73]:
e = OCCUtils.Topo(wire).edges().__next__()

In [74]:
im = BRepMesh_IncrementalMesh(e,0.1)

In [75]:
tess = OCC.Core.Tesselator.ShapeTesselator(e)
tess.Compute(compute_edges=True,mesh_quality=0.1)

In [76]:
tess.ObjGetEdgeCount()

0

In [77]:
tess.ObjGetTriangleCount()

0

# Get edge tesselations from face

In [78]:
tess = OCC.Core.Tesselator.ShapeTesselator(square_pacman_transformed)
tess.Compute(compute_edges=True,mesh_quality=0.1)

In [79]:
tess.ObjGetEdgeCount()

5

In [80]:
tess.ObjEdgeGetVertexCount(1)

2

In [81]:
vertices = []
for i_edge in range(tess.ObjGetEdgeCount()):
    for i_vertex in range(tess.ObjEdgeGetVertexCount(i_edge)):
        if (i_vertex % 2) == 0:
            vertices.append(tess.GetEdgeVertex(i_edge,i_vertex+1))
vertices

[[-50.24996566772461, 43.79387664794922, 1.7763568394002505e-15],
 [3.948725938796997, 7.661407470703125, -4.796163466380676e-14],
 [4.464900970458984, 75.28073120117188, -1.2434497875801753e-14],
 [59.695945739746094, -37.24605941772461, 8.881784197001252e-15],
 [-23.40870475769043, -73.89471435546875, 5.5067062021407764e-14]]

In [82]:
output = a2d.Execute(stock_path,[vertices],callback)

In [83]:
output

In [84]:
mt = BRepBuilderAPI_Transform(path,trsf.Inverted())

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_BRepBuilderAPI_Transform'.
  Possible C/C++ prototypes are:
    BRepBuilderAPI_Transform::BRepBuilderAPI_Transform(gp_Trsf const &)
    BRepBuilderAPI_Transform::BRepBuilderAPI_Transform(TopoDS_Shape const &,gp_Trsf const &,Standard_Boolean const)


In [85]:
renderer.DisplayShape(mt.Shape())

In [86]:
o = output[0]

In [87]:
len(o.AdaptivePaths)

69

In [88]:
len(o.HelixCenterPoint)

2

In [89]:
o.ReturnMotionType

1

In [90]:
o.StartPoint

(50.0, 48.49375)

# Making a class to create adaptive clearing from a face

In [47]:
from OCC.Core.GCE2d import *
from OCC.Core.Geom import *

class Adaptive2d:
    
    def __init__(self,face,ax2=None):
        self.face = face
        if (ax2 is None):
            self.guess_ax()
        else:
            self.ax = ax2
        self.tesselation_mesh_quality = 0.1
        self.z_lift_distance = 1.0
        
    def guess_ax(self):
        f = OCCUtils.face.Face(self.face)
        pln = f.adaptor.Plane()
        self.ax = gp_Ax2(pln.Location(),pln.Axis().Direction(),pln.XAxis().Direction())
        
    def flip_ax_dir(self):
        self.ax.Rotate(gp_Ax1(ax_on_sqp.Location(),ax_on_sqp.XDirection()),pi)
        
    def compute(self):
        trsf = trsf_from_Ax2(self.ax)
        trsf_inv = trsf.Inverted()
        mt = BRepBuilderAPI_Transform(self.face,trsf)
        face_transformed = mt.Shape()
        tess = OCC.Core.Tesselator.ShapeTesselator(face_transformed)
        tess.Compute(compute_edges=True,mesh_quality=self.tesselation_mesh_quality)
        vertices = []
        for i_edge in range(tess.ObjGetEdgeCount()):
            for i_vertex in range(tess.ObjEdgeGetVertexCount(i_edge)):
                if (i_vertex % 2) == 0:
                    vertices.append(tess.GetEdgeVertex(i_edge,i_vertex+1))

        a2d = area.Adaptive2d()
        a2d.tolerance
        a2d.opType
        a2d.stockToLeave
        a2d.toolDiameter = 10
        a2d.helixRampDiameter = 3
        a2d.stepOverFactor = 0.5

        def callback(something):
            return False # True stops processing

        stock_path = [[[0,0],[100,0],[100,100],[0,100]]]

        path = [vertices]
        a2d_output = a2d.Execute(stock_path,path,callback)
        
        comp = TopoDS_Compound()
        builder = BRep_Builder()
        builder.MakeCompound(comp)
        edges = []
        mw = BRepBuilderAPI_MakeWire()
        
        x = 0
        y = 0
        for o_i in a2d_output:
            print("helix center point")
            x, y = o_i.HelixCenterPoint
            
            ax = gp_Ax2(gp_Pnt(x,y,0),gp_Dir(0,0,1),gp_Dir(1,0,0))
            cyl = gp_Cylinder(gp_Ax3(ax),3)

            aLine2d = gp_Lin2d(gp_Pnt2d(0,0),gp_Dir2d(1,1))
            segment = GCE2d_MakeSegment(aLine2d,0,10)
            helixEdge = BRepBuilderAPI_MakeEdge(segment.Value(),Geom_CylindricalSurface(cyl)).Edge()

            #helixEdge.Reverse()

            curve = OCCUtils.edge.Edge(helixEdge).adaptor.Curve()
            v0 = curve.Value(curve.FirstParameter())
            v1 = curve.Value(curve.LastParameter())
            v_now = v0
            
            mw.Add(helixEdge)

            #mw.Add(helixEdge)
            v_now = v1

            for motion_type,path_i in o_i.AdaptivePaths:
                if (motion_type == 1):
                    z = self.z_lift_distance 
                else:
                    z = 0.0
                for x,y in path_i:
                    v_next = gp_Vec(x,y,z)
                    if (v_next - v_now).Magnitude() <= 1e-6:
                        #raise Warning("v:  !")
                        print("skipped a point because |v_next - v_now| <= 1e-6")
                        continue
                    me = BRepBuilderAPI_MakeEdge(gp_Pnt(v_now.XYZ()),gp_Pnt(v_next.XYZ()))
                    mw.Add(me.Edge())
                    v_now = v_next
   



        mt = BRepBuilderAPI_Transform(mw.Wire(),trsf_inv)
        return mt.Shape()
        

In [48]:
hacker_a2d = Adaptive2d(square_pacman)

In [49]:
hacker_a2d.z_lift_distance = 0.0

In [50]:
gp_Vec(hacker_a2d.ax.Direction().XYZ())

< gp_Vec: -3.284198870199983e-17, 0.32556815445715664, -0.9455185755993168, magnitude: 1.0 >

In [56]:
hacker_a2d.flip_ax_dir()

In [57]:
gp_Vec(hacker_a2d.ax.Direction().XYZ())

< gp_Vec: 1.5530666861673513e-16, -0.32556815445715664, 0.9455185755993168, magnitude: 1.0 >

In [ ]:
wire = hacker_a2d.compute()

In [2]:
wire

NameError: name 'wire' is not defined

In [1]:
renderer.DisplayShape(wire)

NameError: name 'renderer' is not defined

# Make a helix

In [ ]:
cyl = gp_Cylinder()